In [1]:
GPU_device_id = str(5)
model_id_save_as = 'learningcurve-daednn-full-final'
architecture_id = 'final-models/learningcurve-dnn-full-final-features'
model_class_id = 'DNN'
testing_dataset_id = '../../source-interdiction/dataset_generation/validation_dataset_full_200keV_log10time_100.npy'
training_dataset_id = '../../source-interdiction/dataset_generation/training_dataset_full_200keV_log10time_10000.npy'
difficulty_setting = 'full'

train_sizes = [50, 100, 500, 1000, 5000, 10000, 15000, 20000,]
earlystop_patience = 10
num_epochs = 2000

In [2]:
import matplotlib.pyplot as plt
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = GPU_device_id


from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, LabelBinarizer
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
import tensorflow as tf
import pickle
import numpy as np
import pandas as pd
from random import choice
from tensorflow.keras.models import load_model

from numpy.random import seed
seed(5)
from tensorflow import set_random_seed
set_random_seed(5)

#### Import model, training function 

In [3]:
from annsa.model_classes import build_dnn_model, compile_model, f1
from annsa.load_dataset import load_easy, load_full, dataset_to_spectrakeys
from annsa.load_pretrained_network import load_features

Using TensorFlow backend.


## Training Data Construction

In [4]:
training_dataset = np.load(training_dataset_id)
training_spectra, training_keys = dataset_to_spectrakeys(training_dataset)

## Load testing dataset

In [5]:
testing_dataset = np.load(testing_dataset_id)
testing_spectra, testing_keys = dataset_to_spectrakeys(testing_dataset)

## Load Model

In [6]:
model_features = load_features(architecture_id)
model_features.input_dim = 1024
model_features.dropout_rate = model_features.dropout_probability
model_features.loss = tf.keras.losses.categorical_crossentropy
model_features.optimizer = tf.keras.optimizers.Adam
model_features.output_function = tf.nn.softmax
model_features.metrics = [f1]
model_features.learning_rate = model_features.learining_rate

In [ ]:
dae_model = load_model('./final-models-keras/daepretrain-full-final.hdf5')

## Train network

# Scale input data

In [8]:
training_spectra_scaled = model_features.scaler.transform(training_spectra)
testing_spectra_scaled = model_features.scaler.transform(testing_spectra)

In [9]:
earlystop_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_f1',
    patience=earlystop_patience,
    mode='max',
    min_delta=0.01,
    restore_best_weights=True)

In [10]:
mlb=LabelBinarizer()

for train_size in train_sizes:
    print('\n\nRunning through training size '+str(train_size))
    k_folds_errors = []

    sss = StratifiedShuffleSplit(n_splits=5, train_size=train_size)
    k = 0
    for train_index, _ in sss.split(training_spectra, training_keys):
        print('Running through fold '+str(k))
        training_keys_binarized = mlb.fit_transform(training_keys.reshape([training_keys.shape[0],1]))
        testing_keys_binarized = mlb.transform(testing_keys)
        
        model = compile_model(
            build_dnn_model,
            model_features)
        model_weights = model.get_weights()
        model_weights_updated = model_weights[:]
        model_weights_updated[0:2] = dae_model.get_weights()[0:2]
        model.set_weights(model_weights_updated)

        csv_logger = tf.keras.callbacks.CSVLogger('./final-models-keras/'+model_id_save_as+'_trainsize'+str(train_size)+'_fold'+str(k)+'.log')

        output = model.fit(
            x=training_spectra_scaled[train_index],
            y=training_keys_binarized[train_index],
            epochs=num_epochs,
            verbose=1,
            validation_data=(testing_spectra_scaled,
                             testing_keys_binarized),
            shuffle=True,
            callbacks=[earlystop_callback, csv_logger],
        )
        model.save('./final-models-keras/'+model_id_save_as+'_trainsize'+str(train_size)+'_fold'+str(k)+'.hdf5')
        k += 1




Running through training size 50
Running through fold 0


/home/ubuntu/anaconda3/envs/tensorflow_p36_update/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1639: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Train on 50 samples, validate on 3000 samples
Epoch 1/2000
50/50 [==============================] - 1s 13ms/sample - loss: 53.4494 - f1: 0.0161 - val_loss: 51.2853 - val_f1: 0.0173
Epoch 2/2000
50/50 [==============================] - 0s 3ms/sample - loss: 51.5896 - f1: 0.0588 - val_loss: 50.8079 - val_f1: 0.0440
Epoch 3/2000
50/50 [==============================] - 0s 3ms/sample - loss: 51.2827 - f1: 0.0345 - val_loss: 50.4561 - val_f1: 0.0437
Epoch 4/2000
50/50 [==============================] - 0s 3ms/sample - loss: 50.7056 - f1: 0.0172 - val_loss: 50.0784 - val_f1: 0.0344
Epoch 5/2000
50/50 [==============================] - 0s 3ms/sample - loss: 50.0853 - f1: 0.0303 - val_loss: 50.1401 - val_f1: 0.0322
Epoch 6/2000
50/50 [==============================] - 0s 3ms/sample - loss: 50.0501 - f1: 0.0631 - val_loss: 50.0303 - val_f1: 0.0395
Epoch 7/2000
50/50 [==============================] - 0s 3ms/sample - loss: 49.3584 - f1: 0.0752 - val_loss: 49.7952 - val_f1: 0.0517
Epoch 8/2000
50

100/100 [==============================] - 0s 2ms/sample - loss: 49.7545 - f1: 0.0082 - val_loss: 49.1219 - val_f1: 0.0434
Epoch 6/2000
100/100 [==============================] - 0s 2ms/sample - loss: 48.1792 - f1: 0.0977 - val_loss: 47.8521 - val_f1: 0.0492
Epoch 7/2000
100/100 [==============================] - 0s 2ms/sample - loss: 47.6602 - f1: 0.0544 - val_loss: 47.6522 - val_f1: 0.0592
Epoch 8/2000
100/100 [==============================] - 0s 2ms/sample - loss: 47.0933 - f1: 0.0358 - val_loss: 47.7683 - val_f1: 0.0594
Epoch 9/2000
100/100 [==============================] - 0s 2ms/sample - loss: 47.1664 - f1: 0.0535 - val_loss: 47.2606 - val_f1: 0.0606
Epoch 10/2000
100/100 [==============================] - 0s 2ms/sample - loss: 46.7578 - f1: 0.0875 - val_loss: 47.1412 - val_f1: 0.0495
Epoch 11/2000
100/100 [==============================] - 0s 2ms/sample - loss: 46.5983 - f1: 0.0598 - val_loss: 46.2849 - val_f1: 0.0435
Epoch 12/2000
100/100 [==============================] - 0s

100/100 [==============================] - 0s 2ms/sample - loss: 45.8819 - f1: 0.0738 - val_loss: 45.6040 - val_f1: 0.0837
Epoch 9/2000
100/100 [==============================] - 0s 2ms/sample - loss: 45.5950 - f1: 0.0548 - val_loss: 45.4595 - val_f1: 0.0959
Epoch 10/2000
100/100 [==============================] - 0s 2ms/sample - loss: 45.3204 - f1: 0.1574 - val_loss: 46.9731 - val_f1: 0.0590
Epoch 11/2000
100/100 [==============================] - 0s 2ms/sample - loss: 45.1404 - f1: 0.1148 - val_loss: 45.0759 - val_f1: 0.0747
Epoch 12/2000
100/100 [==============================] - 0s 2ms/sample - loss: 43.8679 - f1: 0.1033 - val_loss: 43.4927 - val_f1: 0.1493
Epoch 13/2000
100/100 [==============================] - 0s 2ms/sample - loss: 42.7347 - f1: 0.1695 - val_loss: 43.4781 - val_f1: 0.1477
Epoch 14/2000
100/100 [==============================] - 0s 2ms/sample - loss: 42.7977 - f1: 0.1187 - val_loss: 43.0658 - val_f1: 0.1242
Epoch 15/2000
100/100 [==============================] -

...

15000/15000 [==============================] - 2s 139us/sample - loss: 1.0721 - f1: 0.7896 - val_loss: 0.9722 - val_f1: 0.8293
Epoch 43/2000
15000/15000 [==============================] - 2s 138us/sample - loss: 1.0355 - f1: 0.7943 - val_loss: 0.9835 - val_f1: 0.8166
Epoch 44/2000
15000/15000 [==============================] - 2s 143us/sample - loss: 1.0360 - f1: 0.7941 - val_loss: 1.0211 - val_f1: 0.7945
Epoch 45/2000
15000/15000 [==============================] - 2s 137us/sample - loss: 1.0168 - f1: 0.8034 - val_loss: 0.9411 - val_f1: 0.8247
Epoch 46/2000
15000/15000 [==============================] - 2s 137us/sample - loss: 1.0261 - f1: 0.7963 - val_loss: 0.9902 - val_f1: 0.8126
Epoch 47/2000
15000/15000 [==============================] - 2s 137us/sample - loss: 0.9987 - f1: 0.8015 - val_loss: 0.9747 - val_f1: 0.8072
Epoch 48/2000
15000/15000 [==============================] - 2s 142us/sample - loss: 0.9926 - f1: 0.8006 - val_loss: 0.9712 - val_f1: 0.8182
Epoch 49/2000
15000/15000 [

Epoch 29/2000
15000/15000 [==============================] - 2s 139us/sample - loss: 1.3366 - f1: 0.7646 - val_loss: 1.2385 - val_f1: 0.7890
Epoch 30/2000
15000/15000 [==============================] - 2s 140us/sample - loss: 1.2852 - f1: 0.7684 - val_loss: 1.2293 - val_f1: 0.7906
Epoch 31/2000
15000/15000 [==============================] - 2s 140us/sample - loss: 1.2545 - f1: 0.7735 - val_loss: 1.2096 - val_f1: 0.7889
Epoch 32/2000
15000/15000 [==============================] - 2s 139us/sample - loss: 1.2243 - f1: 0.7756 - val_loss: 1.1516 - val_f1: 0.8049
Epoch 33/2000
15000/15000 [==============================] - 2s 139us/sample - loss: 1.2099 - f1: 0.7784 - val_loss: 1.1174 - val_f1: 0.8051
Epoch 34/2000
15000/15000 [==============================] - 2s 140us/sample - loss: 1.1798 - f1: 0.7778 - val_loss: 1.2046 - val_f1: 0.7759
Epoch 35/2000
15000/15000 [==============================] - 2s 140us/sample - loss: 1.1381 - f1: 0.7890 - val_loss: 1.1471 - val_f1: 0.7856
Epoch 36/2000

20000/20000 [==============================] - 3s 136us/sample - loss: 0.9112 - f1: 0.8065 - val_loss: 0.8604 - val_f1: 0.8311
Epoch 47/2000
20000/20000 [==============================] - 3s 136us/sample - loss: 0.8925 - f1: 0.8172 - val_loss: 0.8820 - val_f1: 0.8152
Epoch 48/2000
20000/20000 [==============================] - 3s 136us/sample - loss: 0.8694 - f1: 0.8184 - val_loss: 0.8517 - val_f1: 0.8233
Epoch 49/2000
20000/20000 [==============================] - 3s 135us/sample - loss: 0.8820 - f1: 0.8176 - val_loss: 0.8527 - val_f1: 0.8283
Epoch 50/2000
20000/20000 [==============================] - 3s 138us/sample - loss: 0.8858 - f1: 0.8182 - val_loss: 0.8543 - val_f1: 0.8233
Running through fold 1
Train on 20000 samples, validate on 3000 samples
Epoch 1/2000
20000/20000 [==============================] - 4s 199us/sample - loss: 29.2043 - f1: 0.2760 - val_loss: 17.9611 - val_f1: 0.4336
Epoch 2/2000
20000/20000 [==============================] - 3s 139us/sample - loss: 14.4016 - f

Epoch 54/2000
20000/20000 [==============================] - 3s 139us/sample - loss: 0.8328 - f1: 0.8298 - val_loss: 0.8001 - val_f1: 0.8389
Epoch 55/2000
20000/20000 [==============================] - 3s 138us/sample - loss: 0.8375 - f1: 0.8269 - val_loss: 0.8020 - val_f1: 0.8362
Epoch 56/2000
20000/20000 [==============================] - 3s 138us/sample - loss: 0.8379 - f1: 0.8261 - val_loss: 0.7667 - val_f1: 0.8467
Epoch 57/2000
20000/20000 [==============================] - 3s 138us/sample - loss: 0.8139 - f1: 0.8336 - val_loss: 0.8745 - val_f1: 0.8247
Running through fold 2
Train on 20000 samples, validate on 3000 samples
Epoch 1/2000
20000/20000 [==============================] - 4s 201us/sample - loss: 29.0458 - f1: 0.2627 - val_loss: 17.4453 - val_f1: 0.4747
Epoch 2/2000
20000/20000 [==============================] - 3s 138us/sample - loss: 14.2454 - f1: 0.4297 - val_loss: 11.3231 - val_f1: 0.5020
Epoch 3/2000
20000/20000 [==============================] - 3s 138us/sample - lo

Epoch 29/2000
20000/20000 [==============================] - 3s 140us/sample - loss: 1.1085 - f1: 0.7813 - val_loss: 1.0838 - val_f1: 0.7836
Epoch 30/2000
20000/20000 [==============================] - 3s 140us/sample - loss: 1.0768 - f1: 0.7891 - val_loss: 0.9642 - val_f1: 0.8247
Epoch 31/2000
20000/20000 [==============================] - 3s 140us/sample - loss: 1.0816 - f1: 0.7874 - val_loss: 1.1787 - val_f1: 0.7724
Epoch 32/2000
20000/20000 [==============================] - 3s 140us/sample - loss: 1.0725 - f1: 0.7829 - val_loss: 0.9459 - val_f1: 0.8292
Epoch 33/2000
20000/20000 [==============================] - 3s 140us/sample - loss: 1.0236 - f1: 0.7980 - val_loss: 0.9318 - val_f1: 0.8274
Epoch 34/2000
20000/20000 [==============================] - 3s 139us/sample - loss: 1.0168 - f1: 0.7968 - val_loss: 0.9926 - val_f1: 0.8080
Epoch 35/2000
20000/20000 [==============================] - 3s 139us/sample - loss: 1.0098 - f1: 0.7955 - val_loss: 0.9311 - val_f1: 0.8173
Epoch 36/2000